In [ ]:
import batoid
import danish
import numpy as np
import yaml
from scipy.optimize import root_scalar, least_squares
import asdf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib widget

In [ ]:
# Let's find the footprint of L1 for a specific field angle

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r.yaml")
wavelength=622e-9

In [ ]:
def center_resid(x, thx, optic):
    z = telescope.stopSurface.surface.sag(x, 0) + telescope.stopSurface.coordSys.origin[2]
    vx, vy, vz = batoid.utils.gnomonicToDirCos(np.deg2rad(thx), 0.0)
    ray = batoid.RayVector(x, 0, z, vx, vy, vz, wavelength=wavelength)
    tf = telescope.traceFull(ray)
    x = tf[optic]['out'].x[0]
    return x

In [ ]:
def circle_resid(r, az, cx, thx, optic, target):
    x = np.cos(az)*r+cx
    y = np.sin(az)*r
    z = telescope.stopSurface.surface.sag(x, y) + telescope.stopSurface.coordSys.origin[2]
    vx, vy, vz = batoid.utils.gnomonicToDirCos(np.deg2rad(thx), 0.0)
    ray = batoid.RayVector(x, y, z, vx, vy, vz, wavelength=wavelength)
    tf = telescope.traceFull(ray)
    x = tf[optic]['out'].x[0]
    y = tf[optic]['out'].y[0]
    return np.sqrt(x**2+y**2) - target

In [ ]:
def contour(thx, name, attr, do_center=False):
    # First determine pupil position that passes through center of target optic
    if do_center:
        cx = root_scalar(
            center_resid,
            x0=-4, x1=0,
            args=(thx, name)
        ).root
    else:
        cx = -1

    rs = []
    azs = np.linspace(-np.pi, np.pi, 2*3*5*7, endpoint=False)
    for az in azs:
        args = (az, cx, thx, name, getattr(telescope[name].obscuration.original, attr))
        f0 = circle_resid(0.0, *args)
        f1 = circle_resid(6.5, *args)
        if f0*f1 > 0:
            rs.append(np.nan)
        else:
            rs.append(
                root_scalar(
                    circle_resid,
                    x0=0,
                    x1=6.5,
                    args=args
                ).root
            )
    xs = [r*np.cos(az)+cx for r, az in zip(rs, azs)]
    ys = [r*np.sin(az) for r, az in zip(rs, azs)]
    return xs, ys

In [ ]:
thx = 1.75
plt.figure(figsize=(10, 10))
plt.plot(*contour(thx, 'M1', 'outer'), c='#000000', label='M1')
plt.plot(*contour(thx, 'M1', 'inner', do_center=True), c='#000000', ls=':')
plt.plot(*contour(thx, 'M2', 'outer'), c='#666666', label='M2')
plt.plot(*contour(thx, 'M2', 'inner', do_center=True), c='#666666', ls=':')
plt.plot(*contour(thx, 'M3', 'outer'), c='#BBBBBB', label='M3')
plt.plot(*contour(thx, 'M3', 'inner', do_center=True), c='#BBBBBB', ls=':')
plt.plot(*contour(thx, 'L1_entrance', 'radius'), c='#Ff8179', label='L1')
plt.plot(*contour(thx, 'L1_exit', 'radius'), c='#Ff8179', ls=':')
plt.plot(*contour(thx, 'L2_entrance', 'radius'), c='#Ffc779', label='L2')
plt.plot(*contour(thx, 'L2_exit', 'radius'), c='#Ffc779', ls=':')
plt.plot(*contour(thx, 'Filter_entrance', 'radius'), c='#79b4ff', label='F')
plt.plot(*contour(thx, 'Filter_exit', 'radius'), c='#79b4ff', ls=':')
plt.plot(*contour(thx, 'L3_entrance', 'radius'), c='#D079ff', label='L3')
plt.plot(*contour(thx, 'L3_exit', 'radius'), c='#D079ff', ls=':')
# plt.gca().add_patch(patches.Annulus( (0,0), 4.18, 4.18*0.39, alpha=0.15))
plt.gca().set_aspect('equal')
plt.xlim(-6, 6)
plt.ylim(-6, 6)
plt.legend()

wf = batoid.wavefront(
    telescope,
    theta_x=np.deg2rad(thx),
    theta_y=0.0,
    wavelength=622e-9,
    projection='gnomonic',
    nx=1024,
)

plt.imshow(np.isfinite(wf.array), extent=[-4.18, 4.18, -4.18, 4.18], alpha=0.2, cmap='Reds', vmin=0, vmax=2)

# # Need to actually draw the circle for Filter_entrance
# if thx > fits['Filter_entrance']['thx_min']:
#     radius = np.polyval(fits['Filter_entrance']['pradius'], thx)
#     center = np.polyval(fits['Filter_entrance']['pcenter'], thx)
# print(radius, center)
# plt.gca().add_patch(patches.Circle((center, 0), radius=radius, facecolor='none', edgecolor='green', lw=5))

plt.show()

In [ ]:
wf = batoid.wavefront(
    telescope,
    theta_x=np.deg2rad(1.75),
    theta_y=0.0,
    wavelength=622e-9,
    projection='gnomonic',
    nx=1024,
)
plt.figure()
plt.imshow(wf.array)
plt.show()

In [ ]:
def circle_resid2(
    params, x, y):
    cx, r = params
    res = np.sqrt((np.array(x)-cx)**2 + np.array(y)**2)-r
    w = np.isfinite(res)
    return res[w]

In [ ]:
for name, attr, doc in [
    ('M1', 'outer', False),
    ('M1', 'inner', False),
    ('M2', 'outer', False),
    ('M2', 'inner', False),
    ('M3', 'outer', False),
    ('M3', 'inner', True),
    ('L1_entrance', 'radius', False),
    ('L1_exit', 'radius', False),
    ('L2_entrance', 'radius', False),
    ('L2_exit', 'radius', False),
    ('Filter_entrance', 'radius', False),
    ('Filter_exit', 'radius', False),
    ('L3_entrance', 'radius', False),
    ('L3_exit', 'radius', False),
]:
    print()
    print(name, attr)
    x, y = contour(thx, name, attr, do_center=doc)
    if not np.any(np.isfinite(x)):
        continue
    result = least_squares(circle_resid2, [-1, 5], args=(x, y)).x
    print(result)
    print(np.max(np.abs(circle_resid2(result, x, y)))/8.36*133)
    print(np.median(np.abs(circle_resid2(result, x, y)))/8.36*133)

In [ ]:
# Let's measure from thx=0 to thx=2.0 in steps of 0.01.  Can throw away bad data afterwards.
# Going to save:
# For each thx:
#   For each optic:
#     xs, ys
# Try asdf

In [ ]:
# out = dict()
# thxs = np.linspace(0, 2.0, 201)
# for name, attr, doc in [
#     ('M1', 'outer', False),
#     ('M1', 'inner', True),
#     ('M2', 'outer', False),
#     ('M2', 'inner', True),
#     ('M3', 'outer', False),
#     ('M3', 'inner', True),
#     ('L1_entrance', 'radius', False),
#     ('L1_exit', 'radius', False),
#     ('L2_entrance', 'radius', False),
#     ('L2_exit', 'radius', False),
#     ('Filter_entrance', 'radius', False),
#     ('Filter_exit', 'radius', False),
#     ('L3_entrance', 'radius', False),
#     ('L3_exit', 'radius', False),
# ]:
#     key = name+'_'+attr if attr != 'radius' else name
#     out[key] = dict()
#     results = []
#     for thx in tqdm(thxs):
#         x, y = contour(thx, name, attr, do_center=doc)
#         out[key][thx] = np.array(x), np.array(y)
#     af = asdf.AsdfFile(out)
#     af.write_to('rubin_vignetting_model.asdf')

In [ ]:
out = asdf.open('notebooks/rubin_vignetting_model.asdf')

In [ ]:
thxs = np.array([k for k in out['M1_outer'].keys()])

In [ ]:
names = [
    'M1_outer', 'M1_inner',
    'M2_outer', 'M2_inner',
    'M3_outer', 'M3_inner',
    'L1_entrance', 'L1_exit',
    'L2_entrance', 'L2_exit',
    'Filter_entrance', 'Filter_exit',
    'L3_entrance', 'L3_exit',
]

In [ ]:
# Produce new model.
fits = dict()
for name in names:
    radii = []
    centers = []
    for thx in thxs:
        x, y = out[name][thx]
        r = np.hypot(x, y)
        w = (r < 4.3) & (r > 2.3) & np.isfinite(r)
        if np.sum(w) < 3:
            radius = np.nan
            center = np.nan
        else:
            center, radius = least_squares(circle_resid2, [-1, 5], args=(x[w], y[w])).x
        centers.append(center)
        radii.append(radius)
    centers = np.array(centers)
    radii = np.array(radii)

    # Fit a 3rd order polynomial
    w = np.isfinite(centers)
    pcenter = np.polyfit(thxs[w], centers[w], 3)
    center_fit = np.polyval(pcenter, thxs)
    center_fit[~w] = np.nan

    pradius = np.polyfit(thxs[w], radii[w], 3)
    radius_fit = np.polyval(pradius, thxs)
    radius_fit[~w] = np.nan

    fits[name] = dict(
        thx_min = float(max(np.min(thxs[w])-0.01, 0)),
        pcenter = pcenter.tolist(),
        pradius = pradius.tolist()
    )

    fig, axes = plt.subplots(ncols=2, figsize=(6, 3))
    axes[0].plot(thxs, centers, c='b')
    axes[0].plot(thxs, center_fit, c='m', lw=0.5)
    axes[1].plot(thxs, radii)
    axes[1].plot(thxs, radius_fit, c='m', lw=0.5)
    axes[0].set_ylabel("center (m)")
    axes[1].set_ylabel("radius (m)")
    for ax in axes:
        ax.set_xlabel("th (deg)")
    fig.suptitle(name)
    fig.tight_layout()
    plt.show()